# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab_to_int = dict()
    int_to_vocab = dict()
    
    id = 0
    for word in text:
        if (word not in vocab_to_int):
            vocab_to_int[word] = id
            int_to_vocab[id] = word
            id+=1
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_map_lookup = {'.':'||Period||', ',':'||Comma||', '"':'||Quotation_Mark||', ';':'||Semicolon||', '!':'||Exclamation_mark||', '?':'||Question_mark||', '(':'||Left_Parentheses||',
                       ')':'||Right_Parentheses||', '--':'||Dash||', '\n':'||Return||'}
    
    return token_map_lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32)
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
from tensorflow.python.ops.rnn_cell import BasicLSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #single LSTM cell
    lstm_cell = BasicLSTMCell(rnn_size)

    #multiple LSTM layers
    stacked_rnn_cell = MultiRNNCell([lstm_cell])

    # initialize LSTM cell state
    initial_state = tf.identity(stacked_rnn_cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    
#     num_units = [batch_size, rnn_size]
#     cells = [BasicLSTMCell(num_units=n) for n in num_units]
#     stacked_rnn_cell = MultiRNNCell(cells)
    
#     initial_state = stacked_rnn_cell.zero_state(batch_size, tf.float32)
#     initial_state = tf.identity(initial_state, name = 'initial_state')

    return stacked_rnn_cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    
    SOURCE: Udacity
    """
    # TODO: Implement Function
    
    n_vocab = vocab_size
    n_embedding = embed_dim # Number of embedding features 
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed_input = tf.nn.embedding_lookup(embedding, input_data)
    return embed_input


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(rnn_cell, input_data):
    """
    Create a RNN using a RNN Cell
    :param rnn_cell: RNN Cell
    :param input_data: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    # 'state' is a tensor of shape [batch_size, cell_state_size]
    outputs, final_state = tf.nn.dynamic_rnn(rnn_cell, input_data,
#                                    initial_state=initial_state,
                                   dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    
    SOURCE: Udacity
    """
    # TODO: Implement Function
    embed_input_data = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed_input_data)
    logits = tf.layers.dense(outputs, vocab_size, activation=None)
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def test_get_batches1(get_batches):
    with tf.Graph().as_default():
        test_batch_size = 2
        test_seq_length = 2
        test_int_text = list(range(1, 21))
        batches = get_batches(test_int_text, test_batch_size, test_seq_length)

        # Check type
        assert isinstance(batches, np.ndarray),\
            'Batches is not a Numpy array'

        # Check shape
        assert batches.shape == (5, 2, 2, 2),\
            'Batches returned wrong shape.  Found {}'.format(batches.shape)
    
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    chars_per_batch = batch_size*seq_length
    n_batches = len(int_text)//chars_per_batch
    
    n_chars = n_batches*chars_per_batch

    # Keep only enough characters to make full batches
    input_data = np.array(int_text[:n_chars])
    target_data = np.array(int_text[1:(n_chars+1)])
    target_data[-1] = input_data[0]

    # Reshape into batch_size rows
    inputs = input_data.reshape(batch_size, -1)
    targets = target_data.reshape(batch_size, -1)

    inputs = np.split(inputs, n_batches, -1)
    targets = np.split(targets, n_batches, -1)

    batches = np.array(list(zip(inputs, targets)))
    
#     # Get the number of characters per batch and number of batches we can make
#     chars_per_batch = batch_size * seq_length
#     n_batches = len(int_text)//chars_per_batch
#     # Keep only enough characters to make full batches
#     int_text = int_text[:n_batches * chars_per_batch]
#     # Reshape into batch_size rows
#     int_text = np.asarray(int_text)
#     int_text = int_text.reshape((batch_size, -1))
# #     print(int_text)
    
#     batch_result = []
#     for batch_indx in range(0, n_batches):
#         all_input = []
#         all_target = []
#         inputs_and_targets = []
#         for n in range(0, int_text.shape[1], seq_length):
#             # The features
#             input_each_idx = np.asarray(int_text[:, n:n+seq_length])
#             # The targets, shifted by one
#             target_each_idx_temp = np.asarray(int_text[:, n+1:n+seq_length+1])

#             # For the very last batch, y will be one character short at the end of
#             # the sequences which breaks things. To get around this, I'll make an
#             # array of the appropriate size first, of all zeros, then add the targets.
#             # This will introduce a small artifact in the last batch, but it won't matter.
#             target_each_idx = np.zeros(input_each_idx.shape, dtype=input_each_idx.dtype)
#             target_each_idx[:,:target_each_idx_temp.shape[1]] = target_each_idx_temp

#             all_input.append(input_each_idx)
#             all_target.append(np.asarray(target_each_idx))
        
#         inputs_and_targets.append(all_input)
#         inputs_and_targets.append(all_target)
        
#         print("HI", np.asarray(inputs_and_targets))
#         batch_result.append(inputs_and_targets)
    
#     print(np.asarray(batch_result))
    
    
    
#     num_batches = int((len(int_text)/seq_length)//batch_size)
        
#     # only full batches
#     num_items_per_batch = len(int_text)//(batch_size*seq_length)
#     print(num_items_per_batch)
#     full_batch_end_idx = num_items_per_batch * batch_size
#     int_text = int_text[:full_batch_end_idx]
    
#     batch_result = []
#     for batch_indx in range(0, num_batches):
#         all_input = []
#         all_target = []
#         start_ind = batch_indx*seq_length
#         while start_ind < len(int_text):
#             input_each_idx = int_text[start_ind:start_ind+seq_length]
# #             input_each_idx = ' '.join(str(num) for num in input_each_idx)
#             end_target_idx = (start_ind+seq_length+1) 
            
#             #wrap indices for the target 
#             if end_target_idx > len(int_text):
#                 new_end_idx = -1*(len(int_text)-end_target_idx)
#                 target_each_idx = int_text[start_ind+1:end_target_idx] + int_text[0:new_end_idx]
#             else:
#                 target_each_idx = int_text[start_ind+1:end_target_idx]
# #             target_each_idx = ' '.join(str(num) for num in target_each_idx)
            
#             all_input.append(input_each_idx)
#             all_target.append(target_each_idx)

#             start_ind += num_items_per_batch

#         batch_result.append([all_input, all_target])
    
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test_get_batches1(get_batches)
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [34]:
# Number of Epochs
num_epochs = 150
# Batch Size
batch_size = 60
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 250
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 15

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/57   train_loss = 8.819
Epoch   0 Batch   15/57   train_loss = 6.656
Epoch   0 Batch   30/57   train_loss = 6.355
Epoch   0 Batch   45/57   train_loss = 6.225
Epoch   1 Batch    3/57   train_loss = 6.018
Epoch   1 Batch   18/57   train_loss = 5.988
Epoch   1 Batch   33/57   train_loss = 5.809
Epoch   1 Batch   48/57   train_loss = 5.789
Epoch   2 Batch    6/57   train_loss = 5.594
Epoch   2 Batch   21/57   train_loss = 5.554
Epoch   2 Batch   36/57   train_loss = 5.435
Epoch   2 Batch   51/57   train_loss = 5.409
Epoch   3 Batch    9/57   train_loss = 5.322
Epoch   3 Batch   24/57   train_loss = 5.258
Epoch   3 Batch   39/57   train_loss = 5.299
Epoch   3 Batch   54/57   train_loss = 5.127
Epoch   4 Batch   12/57   train_loss = 5.057
Epoch   4 Batch   27/57   train_loss = 5.269
Epoch   4 Batch   42/57   train_loss = 5.138
Epoch   5 Batch    0/57   train_loss = 4.900
Epoch   5 Batch   15/57   train_loss = 5.019
Epoch   5 Batch   30/57   train_loss = 4.858
Epoch   5 

Epoch  48 Batch    9/57   train_loss = 2.352
Epoch  48 Batch   24/57   train_loss = 2.353
Epoch  48 Batch   39/57   train_loss = 2.319
Epoch  48 Batch   54/57   train_loss = 2.328
Epoch  49 Batch   12/57   train_loss = 2.315
Epoch  49 Batch   27/57   train_loss = 2.335
Epoch  49 Batch   42/57   train_loss = 2.278
Epoch  50 Batch    0/57   train_loss = 2.217
Epoch  50 Batch   15/57   train_loss = 2.309
Epoch  50 Batch   30/57   train_loss = 2.300
Epoch  50 Batch   45/57   train_loss = 2.238
Epoch  51 Batch    3/57   train_loss = 2.173
Epoch  51 Batch   18/57   train_loss = 2.140
Epoch  51 Batch   33/57   train_loss = 2.145
Epoch  51 Batch   48/57   train_loss = 2.213
Epoch  52 Batch    6/57   train_loss = 2.239
Epoch  52 Batch   21/57   train_loss = 2.190
Epoch  52 Batch   36/57   train_loss = 2.174
Epoch  52 Batch   51/57   train_loss = 2.164
Epoch  53 Batch    9/57   train_loss = 2.193
Epoch  53 Batch   24/57   train_loss = 2.186
Epoch  53 Batch   39/57   train_loss = 2.145
Epoch  53 

Epoch  96 Batch   18/57   train_loss = 0.963
Epoch  96 Batch   33/57   train_loss = 0.995
Epoch  96 Batch   48/57   train_loss = 1.020
Epoch  97 Batch    6/57   train_loss = 1.091
Epoch  97 Batch   21/57   train_loss = 1.076
Epoch  97 Batch   36/57   train_loss = 1.078
Epoch  97 Batch   51/57   train_loss = 1.026
Epoch  98 Batch    9/57   train_loss = 1.073
Epoch  98 Batch   24/57   train_loss = 1.081
Epoch  98 Batch   39/57   train_loss = 1.049
Epoch  98 Batch   54/57   train_loss = 1.038
Epoch  99 Batch   12/57   train_loss = 1.062
Epoch  99 Batch   27/57   train_loss = 1.007
Epoch  99 Batch   42/57   train_loss = 1.020
Epoch 100 Batch    0/57   train_loss = 0.928
Epoch 100 Batch   15/57   train_loss = 1.021
Epoch 100 Batch   30/57   train_loss = 1.078
Epoch 100 Batch   45/57   train_loss = 1.061
Epoch 101 Batch    3/57   train_loss = 0.906
Epoch 101 Batch   18/57   train_loss = 0.897
Epoch 101 Batch   33/57   train_loss = 0.916
Epoch 101 Batch   48/57   train_loss = 0.962
Epoch 102 

Epoch 144 Batch   27/57   train_loss = 0.466
Epoch 144 Batch   42/57   train_loss = 0.476
Epoch 145 Batch    0/57   train_loss = 0.438
Epoch 145 Batch   15/57   train_loss = 0.474
Epoch 145 Batch   30/57   train_loss = 0.537
Epoch 145 Batch   45/57   train_loss = 0.494
Epoch 146 Batch    3/57   train_loss = 0.403
Epoch 146 Batch   18/57   train_loss = 0.403
Epoch 146 Batch   33/57   train_loss = 0.423
Epoch 146 Batch   48/57   train_loss = 0.455
Epoch 147 Batch    6/57   train_loss = 0.479
Epoch 147 Batch   21/57   train_loss = 0.489
Epoch 147 Batch   36/57   train_loss = 0.467
Epoch 147 Batch   51/57   train_loss = 0.457
Epoch 148 Batch    9/57   train_loss = 0.485
Epoch 148 Batch   24/57   train_loss = 0.465
Epoch 148 Batch   39/57   train_loss = 0.442
Epoch 148 Batch   54/57   train_loss = 0.461
Epoch 149 Batch   12/57   train_loss = 0.453
Epoch 149 Batch   27/57   train_loss = 0.420
Epoch 149 Batch   42/57   train_loss = 0.440
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [39]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [40]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    idx = np.random.choice(len(probabilities), p=probabilities)
    return int_to_vocab[idx]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [41]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(generously) are you ready to fill with my mother's birthday!
moe_szyslak:(proudly) youuu up here.
seymour_skinner: i understand. now we were your only couch.
moe_szyslak:(" him out) / with my god, you know, but they ain't got is for it.
moe_szyslak: wow, non-losers?! i got nothin' and no more.
kent_brockman: a guy who saw him over on a whim. homie, okay? in how good for? what are you all lookin' at?
moe_szyslak:(uneasy) i guess these days headhunters didn't we go right here.
apu_nahasapeemapetilon:(drunk but smooth) c'mere, a sign.
apu_nahasapeemapetilon:(singing) and dance by our neighbors. nobody loves you. and never done.
young_marge:(upset) hold on... uh, ridin' the. you're one that's gonna be over in some dingy flophouse.


lenny_leonard: hey, i'm worried.
homer_simpson: boy, you guys are inside. you came from sitting there!
crowd:(excited


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.